This code is used to connect a dataset to a MySQL database. The information provided in the code includes:

1. The dataset location file path.
2. The table name in MySQL for the dataset.
3. Details required to connect to the MySQL server, such as the host, user, password, and database name.

import pandas as pd import mysql.connector import os import logging from time import sleep

logging.basicConfig( level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', handlers=[logging.StreamHandler()] )

csv_files = [('Order_Details.csv','Order_Details'), ('List_of_Orders.csv','List_of_Orders'),('Sales_target.csv)','Sales_target']

folder_path = r'D:\Jar company\data Set'

def get_sql_type(dtype): """Map pandas dtypes to MySQL data types.""" if pd.api.types.is_integer_dtype(dtype): return 'BIGINT' elif pd.api.types.is_float_dtype(dtype): return 'DOUBLE' elif pd.api.types.is_bool_dtype(dtype): return 'TINYINT(1)' elif pd.api.types.is_datetime64_any_dtype(dtype): return 'DATETIME' elif dtype.name == 'object': return 'VARCHAR(255)' else: return 'TEXT'

def create_connection(retries=3, delay=5): """Create a connection to the MySQL database with retry mechanism.""" for attempt in range(retries): try: return mysql.connector.connect( host='127.0.0.1', user='amar', password='amar', database='Jar_company' ) except mysql.connector.Error as err: logging.error(f"Connection attempt {attempt + 1} failed: {err}") if attempt < retries - 1: sleep(delay) else: raise

def validate_table_schema(cursor, table_name, df): """Validate the table schema matches the DataFrame.""" cursor.execute(f"DESCRIBE {table_name}") table_schema = {row[0]: row[1] for row in cursor.fetchall()} df_schema = {col: get_sql_type(dtype) for col, dtype in df.dtypes.items()}

for col, sql_type in df_schema.items():
    if col not in table_schema or sql_type not in table_schema[col]:
        logging.warning(f"Schema mismatch for `{table_name}`: `{col}` expected `{sql_type}`, found `{table_schema.get(col, 'MISSING')}`")
try: conn = create_connection() cursor = conn.cursor() logging.info("Database connection established.")

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Check if the file exists
    if not os.path.exists(file_path):
        logging.warning(f"File not found: {file_path}")
        continue

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    df = df.where(pd.notnull(df), None)
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate CREATE TABLE statement
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    
    try:
        cursor.execute(create_table_query)
        validate_table_schema(cursor, table_name, df)

        # Insert data in batches
        batch_size = 1000
        insert_query = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(df.columns))})"
        data = [tuple(None if pd.isna(x) else x for x in row) for _, row in df.iterrows()]

        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            cursor.executemany(insert_query, batch)
            logging.info(f"Inserted batch {i // batch_size + 1} into `{table_name}`.")

        conn.commit()
        logging.info(f"Table `{table_name}` processed successfully.")
    except mysql.connector.Error as err:
        logging.error(f"Error with table `{table_name}`: {err}")
        conn.rollback()
except mysql.connector.Error as conn_err: logging.critical(f"Database connection error: {conn_err}") finally: if 'cursor' in locals() and cursor: cursor.close() if 'conn' in locals() and conn: conn.close() logging.info("Database connection closed.")

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import numpy as np
db = mysql.connector.connect(
        host='127.0.0.1',
        user='amar',
        password='amar',
        database='jar_company'
    )
cur = db.cursor()

The above Code is used to establish a connection between jupyter notebook and SQL

Part 1: Sales and Profitability Analysis

1. Merge the List of Orders and Order Details datasets on the basis of Order ID.
Calculate the total sales (Amount) for each category across all orders.

In [44]:
query = """ select Category, SUM(amount) as Total_Sales from order_details od join list_of_orders o
on o.order_id = od.order_id
group by Category;
        """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Category", "Total Sales"])
df.head()

,Category,Total Sales
0,Furniture,127181
1,Clothing,139054
2,Electronics,165267


2. For each category, calculate the average profit per order and total profit margin
(profit as a percentage of Amount).

In [46]:
query = """ select category, avg(profit) as avg_profit_per_order, (sum(profit) / sum(amount)) * 100 as Total_Profit_Margin
from list_of_orders o join order_details od on o.order_id = od.order_id
group by category; """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Category", "Avg Profit Per Order","Total Profit Margin"])
df

,Category,Avg Profit Per Order,Total Profit Margin
0,Furniture,9.4568,1.8069
1,Clothing,11.7629,8.0278
2,Electronics,34.0714,6.3497


3. Identify the top-performing and underperforming categories based on these
metrics. Also, suggest reasons for their performance differences.

In [48]:
query = """ select category, sum(Profit) as total_profit, (sum(profit) / sum(amount)) * 100 as profit_Margin from order_details 
group by Category
order by Profit_Margin desc;   """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Category", "Total Profit","Profit Margin"])
df

,Category,Total Profit,Profit Margin
0,Clothing,11163,8.0278
1,Electronics,10494,6.3497
2,Furniture,2298,1.8069


1. Clothing is the top performer, with the highest total profit (11,163) and the highest profit margin (8.03%), indicating strong sales efficiency and     profitability.

2. Furniture is the underperformer, with the lowest total profit (2,298) and profit margin (1.81%), suggesting it faces challenges with lower sales or     higher costs impacting profitability.

3. Electronics shows solid performance with decent total profit (10,494) and a profit margin (6.35%), but its margin is lower than Clothing, possibly      due to high production or operational costs.

Part 2: Target Achievement Analysis

1. Using the Sales Target dataset, calculate the percentage change in target sales
   for the Furniture category month-over-month.

In [54]:
query = """ select Category, month_of_Order_Date as Month,Target,
    lag(Target) over (order by Month_of_Order_Date) as Previous_Month_Target,
    ((Target - LAG(Target) over(order by Month_of_Order_Date)) / lag(Target) over(order by Month_of_Order_Date)) * 100 as Monthly_percantage_Change
from Sales_target 
where Category = 'Furniture'  
order by month_of_Order_Date; """

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["Category","Month", "Target"," Previous Month Target","Monthly percentage Change"])
df

,Category,Month,Target,Previous Month Target,Monthly percentage Change
0,Furniture,Apr-18,10400,NaN,None
1,Furniture,Aug-18,10900,10400.0,4.8077
2,Furniture,Dec-18,11400,10900.0,4.5872
3,Furniture,Feb-19,11600,11400.0,1.7544
4,Furniture,Jan-19,11500,11600.0,-0.8621
5,Furniture,Jul-18,10800,11500.0,-6.0870
6,Furniture,Jun-18,10600,10800.0,-1.8519
7,Furniture,Mar-19,11800,10600.0,11.3208
8,Furniture,May-18,10500,11800.0,-11.0169
9,Furniture,Nov-18,11300,10500.0,7.6190


Using the LAG window function, the previous month's target was calculated, and the monthly percentage change was computed as: (Target - Previous Month Target) / Previous Month Target.

2. Analyse the trends to identify months with significant target fluctuations.Suggest strategies for aligning target expectations with actual              performance trends.

Analysis of Target Fluctuations:

1. Identify Significant Positive Fluctuations:
   Mar-19 (+11.32%) had the highest increase in target, likely due to a surge in demand or seasonal factors.
   Aug-18 (+4.81%) and Dec-18 (+4.59%) show moderate positive changes, reflecting possible promotional efforts or demand upticks.

2. Detect Significant Negative Fluctuations:
   May-18 (-11.02%) had the steepest decline in targets, suggesting overestimation in prior months or seasonal drops.
   Jul-18 (-6.09%) also shows a notable decrease, indicating potential demand volatility.

3. Observe Stable Changes:
   Months like Jan-19 (-0.86%), Sep-18 (-0.90%), and Oct-18 (-1.77%) exhibit small changes, signaling stable target expectations.
   Strategies for Aligning Target Expectations:


4. Utilize Historical Trends:
   Analyze historical data to forecast realistic targets, reducing large and sudden fluctuations.

5. Incorporate Seasonality:
   Adjust targets based on seasonal trends, as seen in Mar-19 and Dec-18, to align with demand patterns.

6. Set Incremental Adjustments:
   Avoid sharp drops like May-18 (-11.02%) by gradually adjusting targets to maintain motivation and consistency.

7. Compare with Actual Performance:
   Regularly monitor actual sales against targets to fine-tune expectations and ensure targets remain achievable.

8. Collaborate Across Teams:
   Work closely with sales and marketing teams to gather insights and set practical, data-backed targets.

9. Build Flexibility into Planning:
   Allow for adjustments to targets based on changing market conditions or unforeseen trends to stay competitive.

Part 3: Regional Performance Insights

1. From the List of Orders dataset, identify the top 5 states with the highest order count. For each of these states, calculate the total sales and    average profit.

In [119]:

query = """ with  top_states as (
select state, count(order_id) as Order_Count
from List_of_orders
group by state
order by Order_Count desc
limit 5
)
select
    lo.State,
    count(lo.order_id) as Total_Orders,
    sum(od.amount) as Total_Sales,
    avg(od.profit) as Average_Profit
from List_of_orders lo
join Order_details od on lo.order_id = od.order_id
where lo.State in (select State from Top_States)
group by lo.State
order by Total_Orders DESC; """

cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data, columns = ["States","Total Orders", "Total Sales"," Average Profit"])
df

,States,Total Orders,Total Sales,Average Profit
0,Madhya Pradesh,340,105140,16.3265
1,Maharashtra,290,95348,21.2966
2,Gujarat,87,21058,5.3448
3,Rajasthan,74,21149,16.9865
4,Punjab,60,16786,-10.1500
